In [ ]:
%reset -f

In [ ]:
import os
os.chdir(r'/home')
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import scipy
import torch.nn as nn
import torch.nn.functional as F
from timeit import default_timer
from torch.utils.data import TensorDataset, DataLoader, random_split

# GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

# Functions
def derivee(y,x):
  return torch.autograd.grad(y,x,torch.ones_like(y),retain_graph=True,create_graph=True)[0]

def som(s):
  return torch.sum(s,dim=-1,keepdim=True)

namedata = 'Struct_4_256'
Data = scipy.io.loadmat(namedata+'.mat')
vox = torch.tensor(Data['vox'], dtype = torch.float32).to(device)

In [ ]:
# NN module
class MLP_gelu(nn.Module):
    def __init__(self,insize,outsize,width,layers):
        super().__init__()
        self.input_layer = nn.Linear(insize,width)
        self.hidden_layers = nn.ModuleList([])
        self.output_layer = nn.Linear(width,outsize)
        for i in range(layers):
          self.linear_layer = nn.Linear(width,width)
          self.hidden_layers.append(self.linear_layer)
        self.layers = layers

    def forward(self, x):
        layer_out = F.gelu(self.input_layer(x))
        for i in range(self.layers):
            layer_out = F.gelu(self.hidden_layers[i](layer_out)) + layer_out
        layer_out = self.output_layer(layer_out)
        return layer_out

# Neural network in PINN
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.NN_trunk = MLP_gelu(insize=6,outsize=3,width=64,layers=4)

    def forward(self, x, y, z, e):
        # Normalisation
        scaler = torch.max(torch.abs(e))
        e = e/scaler
        # NN modulus
        U_tilde = self.NN_trunk(torch.cat([torch.sin(x * 2*torch.pi),
                                     torch.cos(x * 2*torch.pi),
                                     torch.sin(y * 2*torch.pi),
                                     torch.cos(y * 2*torch.pi),
                                     torch.sin(z * 2*torch.pi),
                                     torch.cos(z * 2*torch.pi)], dim=-1))
        ux_tilde, uy_tilde, uz_tilde = U_tilde[...,0:1], U_tilde[...,1:2], U_tilde[...,2:3]
        # Periodic Boundary condition
        ux = (ux_tilde + e[0]*x + e[3]*y + e[4]*z)*scaler
        uy = (uy_tilde + e[3]*x + e[1]*y + e[5]*z)*scaler
        uz = (uz_tilde + e[4]*x + e[5]*y + e[2]*z)*scaler
        return ux, uy, uz

class Homogenisation(nn.Module):
    def __init__(self, struct, epsilon_bar, batch_size=10000):
        super(Homogenisation, self).__init__()

        # Configurations
        self.L = 1.0 # side length of the domain
        self.struct = struct # meta structure
        self.iter = 0
        self.batch_size = batch_size
        # Average Strain
        self.epsilon_bar = epsilon_bar
        # Constitutive Model
        self.E, self.nu = 10.0, 0.3
        # Solution Space (Displacement)
        self.U = PINN().to(device)
        # Coordinates
        Ndim = len(struct)
        X, Y, Z = torch.meshgrid(torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                       torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                       torch.linspace(-0.5+0.5/Ndim,0.5-0.5/Ndim,Ndim),
                       indexing='ij')
        L_pde = (self.struct.reshape(-1) == 1).to(device)
        self.x_pde = X.reshape(-1)[:,None].to(device)[L_pde]
        self.y_pde = Y.reshape(-1)[:,None].to(device)[L_pde]
        self.z_pde = Z.reshape(-1)[:,None].to(device)[L_pde]
        self.x_pde.requires_grad = True
        self.y_pde.requires_grad = True
        self.z_pde.requires_grad = True
        self.N = self.x_pde.shape[0]
        # Iteration
        self.iter = 0
        self.scaler = torch.max(torch.abs(epsilon_bar))

    def batch_backward(self):
        energy = 0.0
        for i in range(0, self.N, self.batch_size):
            end_idx = min(i + self.batch_size, self.N)
            x_batch = self.x_pde[i:end_idx]
            y_batch = self.y_pde[i:end_idx]
            z_batch = self.z_pde[i:end_idx]
            batch_energy = self.Psi(x_batch, y_batch, z_batch) / self.N
            batch_energy.backward()  # accumulate gradients
            energy += batch_energy.item()
        return energy

    def Psi(self, x, y, z):  # strain Energy (linear Elasticity) in 3D
        # Displacement
        ux, uy, uz = self.U(x, y, z, self.epsilon_bar)
        # Strain
        epsilon_xx = derivee(ux, x)
        epsilon_yy = derivee(uy, y)
        epsilon_zz = derivee(uz, z)
        epsilon_xy = 0.5 * (derivee(ux, y) + derivee(uy, x))
        epsilon_xz = 0.5 * (derivee(ux, z) + derivee(uz, x))
        epsilon_yz = 0.5 * (derivee(uy, z) + derivee(uz, y))
        # Constitutive
        pref = self.E / ((1.0 + self.nu) * (1.0 - 2.0 * self.nu))
        G = self.E / (2.0 * (1.0 + self.nu))
        # Stress
        sigma_xx = pref * ((1.0 - self.nu) * epsilon_xx + self.nu * (epsilon_yy + epsilon_zz))
        sigma_yy = pref * ((1.0 - self.nu) * epsilon_yy + self.nu * (epsilon_xx + epsilon_zz))
        sigma_zz = pref * ((1.0 - self.nu) * epsilon_zz + self.nu * (epsilon_xx + epsilon_yy))
        sigma_xy = 2.0 * G * epsilon_xy
        sigma_xz = 2.0 * G * epsilon_xz
        sigma_yz = 2.0 * G * epsilon_yz
        # Strain energy density (3D)
        Psi = 0.5 * (epsilon_xx*sigma_xx + epsilon_yy*sigma_yy + epsilon_zz*sigma_zz +
                     2.0 * (epsilon_xy*sigma_xy + epsilon_xz*sigma_xz + epsilon_yz*sigma_yz)) / self.scaler**2
        return Psi.sum()

    def Solution(self): # Calculate stress field in 3D with batching
        U_list = []
        epsilon_list = []
        sigma_list = []
        for i in range(0, self.N, self.batch_size):
            end_idx = min(i + self.batch_size, self.N)
            x_batch = self.x_pde[i:end_idx]
            y_batch = self.y_pde[i:end_idx]
            z_batch = self.z_pde[i:end_idx]
            # Displacement
            ux, uy, uz = self.U(x_batch, y_batch, z_batch, self.epsilon_bar)
            # Strain
            epsilon_xx = derivee(ux, x_batch)
            epsilon_yy = derivee(uy, y_batch)
            epsilon_zz = derivee(uz, z_batch)
            epsilon_xy = (derivee(ux, y_batch) + derivee(uy, x_batch)) / 2.0
            epsilon_xz = (derivee(ux, z_batch) + derivee(uz, x_batch)) / 2.0
            epsilon_yz = (derivee(uy, z_batch) + derivee(uz, y_batch)) / 2.0
            # Constitutive
            pref = self.E / ((1.0 + self.nu) * (1.0 - 2.0 * self.nu))
            G = self.E / (2.0 * (1.0 + self.nu))
            # Stress
            sigma_xx = pref * ((1.0 - self.nu) * epsilon_xx + self.nu * (epsilon_yy + epsilon_zz))
            sigma_yy = pref * ((1.0 - self.nu) * epsilon_yy + self.nu * (epsilon_xx + epsilon_zz))
            sigma_zz = pref * ((1.0 - self.nu) * epsilon_zz + self.nu * (epsilon_xx + epsilon_yy))
            sigma_xy = 2.0 * G * epsilon_xy
            sigma_xz = 2.0 * G * epsilon_xz
            sigma_yz = 2.0 * G * epsilon_yz
            # Assemble
            U_batch = torch.cat((ux, uy, uz), dim=-1)
            Epsilon_batch = torch.cat((epsilon_xx, epsilon_yy, epsilon_zz,
                                    epsilon_xy, epsilon_xz, epsilon_yz), dim=-1)
            Sigma_batch = torch.cat((sigma_xx, sigma_yy, sigma_zz,
                                    sigma_xy, sigma_xz, sigma_yz), dim=-1)
            U_list.append(U_batch)
            epsilon_list.append(Epsilon_batch)
            sigma_list.append(Sigma_batch)
        U = torch.cat(U_list, dim=0)
        U = U-torch.mean(U,dim=0,keepdim=True)
        Epsilon = torch.cat(epsilon_list, dim=0)
        Sigma = torch.cat(sigma_list, dim=0)
        return U, Epsilon, Sigma

In [ ]:
E_bar = torch.tensor([1.0,0.0,0.0,0.0,0.0,0.0]).to(device)
model = Homogenisation(vox,E_bar,batch_size=1000000)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for ep in range(1000):
    model.train()
    optimizer.zero_grad()
    Loss = model.batch_backward()
    optimizer.step()
    if (ep+2) % 100 == 1:
        print(f'Epoch:{ep+1}, PDE Loss:{Loss:.16e}')


In [ ]:
model.U.eval()
U, Epsilon, Sigma = model.Solution()

x = model.x_pde.detach().cpu().numpy()
y = model.y_pde.detach().cpu().numpy()
z = model.z_pde.detach().cpu().numpy()
ux, uy, uz = U[:, 0].detach().cpu().numpy(), U[:, 1].detach().cpu().numpy(), U[:, 2].detach().cpu().numpy()

fig = plt.figure(figsize=(8.5, 3))
ax1 = fig.add_subplot(131, projection='3d')
ax2 = fig.add_subplot(132, projection='3d')
ax3 = fig.add_subplot(133, projection='3d')
ax1.set_title('u_x'); ax1.axis('off')
ax2.set_title('u_y'); ax2.axis('off')
ax3.set_title('u_z'); ax3.axis('off')
sc1 = ax1.scatter(x, y, z, c=ux, cmap='jet', s=0.1)
sc2 = ax2.scatter(x, y, z, c=uy, cmap='jet', s=0.1)
sc3 = ax3.scatter(x, y, z, c=uz, cmap='jet', s=0.1)
plt.colorbar(sc1, ax=ax1, shrink=0.5, pad=0.1)
plt.colorbar(sc2, ax=ax2, shrink=0.5, pad=0.1)
plt.colorbar(sc3, ax=ax3, shrink=0.5, pad=0.1)
plt.tight_layout()
plt.show()